In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

In [ ]:
# Need config.json file in the same folder
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
ml_client = MLClient.from_config(credential=DefaultAzureCredential(exclude_shared_token_cache_credential=True),
                     logging_enable=True)

### Retrieve diabetes dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
 
# Load diabetes datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target
 
# Create a pandas DataFrame that serves as sample input for the deployed ElasticNet model
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
query = data.drop(["progression"], axis=1)

In [ ]:
data.to_csv("diabetes_data.csv", index=False)
query.to_csv("diabetes_query.csv", index=False)

## Register dataset

In [ ]:
data = Data(
    path = "./diabetes_data.csv",
    type = AssetTypes.URI_FILE,
    description = "diabades data for oh4ml",
    name = "diabetes_data_oh4ml",
    version = '1'
)

ml_client.data.create_or_update(data)

In [ ]:
data = Data(
    path = "./diabetes_query.csv",
    type = AssetTypes.URI_FILE,
    description = "diabetes query for oh4ml",
    name = "diabetes_query_oh4ml",
    version = '1'
)

ml_client.data.create_or_update(data)

## Train local

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
regression_model = Ridge(alpha=0.03)
regression_model.fit(X_train, y_train)
prediction = regression_model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, prediction)

print("MSE: ", mse)
print("RMSE: ", rmse)
print("R2: ", r2)

In [ ]:
fig = plt.figure()
idx = np.argsort(y_test)
plt.plot(y_test[idx], prediction[idx])